In [46]:
%pip install -q jsonlines

import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.model_monitor import (
    DefaultModelMonitor,
    DataCaptureConfig,
    CronExpressionGenerator,
    ModelQualityMonitor,
    EndpointInput,
)
from sagemaker.model_monitor.dataset_format import DatasetFormat
import json
import jsonlines
import utils
from importlib import reload
from tqdm import trange
import numpy as np
import time
import pandas as pd
from time import gmtime, strftime

reload(utils)

Note: you may need to restart the kernel to use updated packages.


<module 'utils' from '/home/sagemaker-user/src/monitoring/utils.py'>

In [4]:
sm_session = sagemaker.Session()

endpoint_name = "index-predictor-endpoint"
feature_group_name = "index-predictor-feature-group-v7"
bucket_name = "team1-index-predictor-bucket"
data_version = "2024-06-26-09-33"

data_capture_prefix = "data-capture"
data_capture_s3_url = f"s3://{bucket_name}/{data_capture_prefix}"

In [5]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
)

### Downloading data from feature store

In [30]:
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sm_session)

query = feature_group.athena_query()

query.run(
    query_string=f"""SELECT * FROM "{query.table_name}" WHERE version = '{data_version}'""",
    output_location=f"s3://{bucket_name}/model_monitor/data/",
)

query.wait()

df = query.as_dataframe()

train_df = df[df["type"] == "train"].copy()
validation_df = df[df["type"] == "validation"].copy()
test_df = df[df["type"] == "test"].copy()

selected_test_df = test_df.copy().sample(n=100, random_state=1)

In [31]:
columns_to_drop = ["type", "version", "write_time", "api_invocation_time", "is_deleted", "datetime", "close_target"]

df.drop(
    columns=columns_to_drop,
    inplace=True,
)
train_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
validation_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
selected_test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)

In [47]:
train_df.to_csv('/tmp/train_df.csv', index=False, header=False)

train_df_s3_uri = S3Uploader.upload('/tmp/train_df.csv', f's3://{bucket_name}/model_monitor/train_df.csv')

In [32]:
def generate_endpoint_traffic(predictor, data):
    ln = len(data)
    for i in trange(ln):
        row_data = data.iloc[i].values.reshape(1, -1)
        predictions = np.array(
            predictor.predict(row_data), dtype=float
        ).squeeze()

In [33]:
generate_endpoint_traffic(predictor, selected_test_df)

100%|██████████| 100/100 [00:01<00:00, 66.88it/s]


In [34]:
capture_files = utils.get_file_list(bucket_name, data_capture_prefix)

Found 8 files in s3://team1-index-predictor-bucket/data-capture


In [44]:
data_monitor = DefaultModelMonitor(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=10,
    max_runtime_in_seconds=3600,
    sagemaker_session=sm_session,
)

In [49]:
data_baseline_job_name = f"model-monitor-baseline-{strftime('%d-%H-%M-%S', gmtime())}"

data_baseline_job = data_monitor.suggest_baseline(
    baseline_dataset=train_df_s3_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=f"s3://{bucket_name}/model_monitor/baseline_results",
    wait=False,
    logs=False,
    job_name=data_baseline_job_name,
)

INFO:sagemaker:Creating processing-job with name model-monitor-baseline-26-19-08-36


In [51]:
data_baseline_job.wait(logs=False)
data_monitor.describe_latest_baselining_job()

!

{'ProcessingInputs': [{'InputName': 'baseline_dataset_input',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://team1-index-predictor-bucket/model_monitor/train_df.csv/train_df.csv',
    'LocalPath': '/opt/ml/processing/input/baseline_dataset_input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'monitoring_output',
    'S3Output': {'S3Uri': 's3://team1-index-predictor-bucket/model_monitor/baseline_results',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False}]},
 'ProcessingJobName': 'model-monitor-baseline-26-19-08-36',
 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1,
   'InstanceType': 'ml.m5.xlarge',
   'VolumeSizeInGB': 10}},
 'StoppingCondition': {'MaxRuntimeInSeconds': 3600},
 'AppSpecification': {'ImageUri': '048819808253.dkr.ecr.eu-central-1.amazonaws.com/sag

In [62]:
!aws s3 cp s3://{bucket_name}/model_monitor/baseline_results/constraints.json ./tmp/
!aws s3 cp s3://{bucket_name}/model_monitor/baseline_results/statistics.json ./tmp/
!head -15 tmp/constraints.json
!head -15 tmp/statistics.json

download: s3://team1-index-predictor-bucket/model_monitor/baseline_results/constraints.json to tmp/constraints.json
download: s3://team1-index-predictor-bucket/model_monitor/baseline_results/statistics.json to tmp/statistics.json
{
  "version" : 0.0,
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "_c1",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
{
  "version" : 0.0,
  "dataset" : {
    "item_count" : 6908
  },
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Fractional",
    "numerical_statistics" : {
      "common" : {
        "num_present" : 6908,
        "num_missing" : 0
      },
      "mean" : 5362.103575635247,
      "sum" : 3.704141150048828E7,


In [92]:
latest_data_capture_s3_url = utils.get_latest_data_capture_s3_url(bucket_name, data_capture_prefix)

utils.run_model_monitor_job(
    region="eu-central-1",
    instance_type="ml.m5.xlarge",
    role=sagemaker.get_execution_role(),
    data_capture_path=latest_data_capture_s3_url,
    statistics_path=f"s3://{bucket_name}/model_monitor/baseline_results/statistics.json",
    constraints_path=f"s3://{bucket_name}/model_monitor/baseline_results/constraints.json",
    reports_path=f"s3://{bucket_name}/model_monitor/baseline_results/reports/",
    preprocessor_path="./record_preprocessor.py",
    instance_count=1,
    postprocessor_path=None,
    publish_cloudwatch_metrics="Disabled",
    logs=False,
)

Found 11 files in s3://team1-index-predictor-bucket/data-capture
Latest data capture S3 url: s3://team1-index-predictor-bucket/data-capture/index-predictor-endpoint/AllTraffic/2024/06/26/19


INFO:sagemaker:Creating processing-job with name sagemaker-model-monitor-analyzer-2024-06-26-20-55-01-432


...........................................................!

In [94]:
manual_monitoring_job_output_s3_url = utils.get_latest_monitoring_report_s3_url("sagemaker-model-monitor-analyzer")

Latest monitoring report S3 url: s3://team1-index-predictor-bucket/model_monitor/baseline_results/reports//9


In [95]:
!aws s3 ls {manual_monitoring_job_output_s3_url}/

2024-06-26 20:58:55      37229 constraint_violations.json
2024-06-26 20:58:49      24989 constraints.json
2024-06-26 20:58:50     277283 statistics.json


In [103]:
data_mon_schedule_name = "index-predictor-data-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

data_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_mon_schedule_name,
    endpoint_input=predictor.endpoint_name,
    record_preprocessor_script="./record_preprocessor.py",
    output_s3_uri=f"s3://{bucket_name}/model_monitor/baseline_results/reports/",
    statistics=data_monitor.baseline_statistics(),
    constraints=data_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: index-predictor-data-monitor-schedule-2024-06-26-21-24-56


In [108]:
while data_monitor.describe_schedule()["MonitoringScheduleStatus"] != "Scheduled":
    print(f"Waiting until data monitoring schedule status becomes Scheduled")
    time.sleep(3)

data_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:eu-central-1:567821811420:monitoring-schedule/index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
 'MonitoringScheduleName': 'index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2024, 6, 26, 21, 24, 57, 850000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 6, 26, 23, 12, 21, 239000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2024-06-26-21-24-56-894',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'index-predictor-endpoint',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
  'ScheduledTime': datetime.datetime(2024, 6, 26, 23, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 6, 2

In [ ]:
data_monitor.delete_monitoring_schedule(name=data_mon_schedule_name)

In [46]:
%pip install -q jsonlines

import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.model_monitor import (
    DefaultModelMonitor,
    DataCaptureConfig,
    CronExpressionGenerator,
    ModelQualityMonitor,
    EndpointInput,
)
from sagemaker.model_monitor.dataset_format import DatasetFormat
import json
import jsonlines
import utils
from importlib import reload
from tqdm import trange
import numpy as np
import time
import pandas as pd
from time import gmtime, strftime

reload(utils)

Note: you may need to restart the kernel to use updated packages.


<module 'utils' from '/home/sagemaker-user/src/monitoring/utils.py'>

In [4]:
sm_session = sagemaker.Session()

endpoint_name = "index-predictor-endpoint"
feature_group_name = "index-predictor-feature-group-v7"
bucket_name = "team1-index-predictor-bucket"
data_version = "2024-06-26-09-33"

data_capture_prefix = "data-capture"
data_capture_s3_url = f"s3://{bucket_name}/{data_capture_prefix}"

In [5]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
)

### Downloading data from feature store

In [30]:
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sm_session)

query = feature_group.athena_query()

query.run(
    query_string=f"""SELECT * FROM "{query.table_name}" WHERE version = '{data_version}'""",
    output_location=f"s3://{bucket_name}/model_monitor/data/",
)

query.wait()

df = query.as_dataframe()

train_df = df[df["type"] == "train"].copy()
validation_df = df[df["type"] == "validation"].copy()
test_df = df[df["type"] == "test"].copy()

selected_test_df = test_df.copy().sample(n=100, random_state=1)

In [31]:
columns_to_drop = ["type", "version", "write_time", "api_invocation_time", "is_deleted", "datetime", "close_target"]

df.drop(
    columns=columns_to_drop,
    inplace=True,
)
train_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
validation_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
selected_test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)

In [47]:
train_df.to_csv('/tmp/train_df.csv', index=False, header=False)

train_df_s3_uri = S3Uploader.upload('/tmp/train_df.csv', f's3://{bucket_name}/model_monitor/train_df.csv')

In [32]:
def generate_endpoint_traffic(predictor, data):
    ln = len(data)
    for i in trange(ln):
        row_data = data.iloc[i].values.reshape(1, -1)
        predictions = np.array(
            predictor.predict(row_data), dtype=float
        ).squeeze()

In [33]:
generate_endpoint_traffic(predictor, selected_test_df)

100%|██████████| 100/100 [00:01<00:00, 66.88it/s]


In [34]:
capture_files = utils.get_file_list(bucket_name, data_capture_prefix)

Found 8 files in s3://team1-index-predictor-bucket/data-capture


In [44]:
data_monitor = DefaultModelMonitor(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=10,
    max_runtime_in_seconds=3600,
    sagemaker_session=sm_session,
)

In [49]:
data_baseline_job_name = f"model-monitor-baseline-{strftime('%d-%H-%M-%S', gmtime())}"

data_baseline_job = data_monitor.suggest_baseline(
    baseline_dataset=train_df_s3_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=f"s3://{bucket_name}/model_monitor/baseline_results",
    wait=False,
    logs=False,
    job_name=data_baseline_job_name,
)

INFO:sagemaker:Creating processing-job with name model-monitor-baseline-26-19-08-36


In [51]:
data_baseline_job.wait(logs=False)
data_monitor.describe_latest_baselining_job()

!

{'ProcessingInputs': [{'InputName': 'baseline_dataset_input',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://team1-index-predictor-bucket/model_monitor/train_df.csv/train_df.csv',
    'LocalPath': '/opt/ml/processing/input/baseline_dataset_input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'monitoring_output',
    'S3Output': {'S3Uri': 's3://team1-index-predictor-bucket/model_monitor/baseline_results',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False}]},
 'ProcessingJobName': 'model-monitor-baseline-26-19-08-36',
 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1,
   'InstanceType': 'ml.m5.xlarge',
   'VolumeSizeInGB': 10}},
 'StoppingCondition': {'MaxRuntimeInSeconds': 3600},
 'AppSpecification': {'ImageUri': '048819808253.dkr.ecr.eu-central-1.amazonaws.com/sag

In [62]:
!aws s3 cp s3://{bucket_name}/model_monitor/baseline_results/constraints.json ./tmp/
!aws s3 cp s3://{bucket_name}/model_monitor/baseline_results/statistics.json ./tmp/
!head -15 tmp/constraints.json
!head -15 tmp/statistics.json

download: s3://team1-index-predictor-bucket/model_monitor/baseline_results/constraints.json to tmp/constraints.json
download: s3://team1-index-predictor-bucket/model_monitor/baseline_results/statistics.json to tmp/statistics.json
{
  "version" : 0.0,
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "_c1",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
{
  "version" : 0.0,
  "dataset" : {
    "item_count" : 6908
  },
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Fractional",
    "numerical_statistics" : {
      "common" : {
        "num_present" : 6908,
        "num_missing" : 0
      },
      "mean" : 5362.103575635247,
      "sum" : 3.704141150048828E7,


In [92]:
latest_data_capture_s3_url = utils.get_latest_data_capture_s3_url(bucket_name, data_capture_prefix)

utils.run_model_monitor_job(
    region="eu-central-1",
    instance_type="ml.m5.xlarge",
    role=sagemaker.get_execution_role(),
    data_capture_path=latest_data_capture_s3_url,
    statistics_path=f"s3://{bucket_name}/model_monitor/baseline_results/statistics.json",
    constraints_path=f"s3://{bucket_name}/model_monitor/baseline_results/constraints.json",
    reports_path=f"s3://{bucket_name}/model_monitor/baseline_results/reports/",
    preprocessor_path="./record_preprocessor.py",
    instance_count=1,
    postprocessor_path=None,
    publish_cloudwatch_metrics="Disabled",
    logs=False,
)

Found 11 files in s3://team1-index-predictor-bucket/data-capture
Latest data capture S3 url: s3://team1-index-predictor-bucket/data-capture/index-predictor-endpoint/AllTraffic/2024/06/26/19


INFO:sagemaker:Creating processing-job with name sagemaker-model-monitor-analyzer-2024-06-26-20-55-01-432


...........................................................!

In [94]:
manual_monitoring_job_output_s3_url = utils.get_latest_monitoring_report_s3_url("sagemaker-model-monitor-analyzer")

Latest monitoring report S3 url: s3://team1-index-predictor-bucket/model_monitor/baseline_results/reports//9


In [95]:
!aws s3 ls {manual_monitoring_job_output_s3_url}/

2024-06-26 20:58:55      37229 constraint_violations.json
2024-06-26 20:58:49      24989 constraints.json
2024-06-26 20:58:50     277283 statistics.json


In [103]:
data_mon_schedule_name = "index-predictor-data-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

data_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_mon_schedule_name,
    endpoint_input=predictor.endpoint_name,
    record_preprocessor_script="./record_preprocessor.py",
    output_s3_uri=f"s3://{bucket_name}/model_monitor/baseline_results/reports/",
    statistics=data_monitor.baseline_statistics(),
    constraints=data_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: index-predictor-data-monitor-schedule-2024-06-26-21-24-56


In [108]:
while data_monitor.describe_schedule()["MonitoringScheduleStatus"] != "Scheduled":
    print(f"Waiting until data monitoring schedule status becomes Scheduled")
    time.sleep(3)

data_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:eu-central-1:567821811420:monitoring-schedule/index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
 'MonitoringScheduleName': 'index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2024, 6, 26, 21, 24, 57, 850000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 6, 26, 23, 12, 21, 239000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2024-06-26-21-24-56-894',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'index-predictor-endpoint',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
  'ScheduledTime': datetime.datetime(2024, 6, 26, 23, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 6, 2

In [ ]:
data_monitor.delete_monitoring_schedule(name=data_mon_schedule_name)

In [46]:
%pip install -q jsonlines

import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.model_monitor import (
    DefaultModelMonitor,
    DataCaptureConfig,
    CronExpressionGenerator,
    ModelQualityMonitor,
    EndpointInput,
)
from sagemaker.model_monitor.dataset_format import DatasetFormat
import json
import jsonlines
import utils
from importlib import reload
from tqdm import trange
import numpy as np
import time
import pandas as pd
from time import gmtime, strftime

reload(utils)

Note: you may need to restart the kernel to use updated packages.


<module 'utils' from '/home/sagemaker-user/src/monitoring/utils.py'>

In [4]:
sm_session = sagemaker.Session()

endpoint_name = "index-predictor-endpoint"
feature_group_name = "index-predictor-feature-group-v7"
bucket_name = "team1-index-predictor-bucket"
data_version = "2024-06-26-09-33"

data_capture_prefix = "data-capture"
data_capture_s3_url = f"s3://{bucket_name}/{data_capture_prefix}"

In [5]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
)

### Downloading data from feature store

In [30]:
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sm_session)

query = feature_group.athena_query()

query.run(
    query_string=f"""SELECT * FROM "{query.table_name}" WHERE version = '{data_version}'""",
    output_location=f"s3://{bucket_name}/model_monitor/data/",
)

query.wait()

df = query.as_dataframe()

train_df = df[df["type"] == "train"].copy()
validation_df = df[df["type"] == "validation"].copy()
test_df = df[df["type"] == "test"].copy()

selected_test_df = test_df.copy().sample(n=100, random_state=1)

In [31]:
columns_to_drop = ["type", "version", "write_time", "api_invocation_time", "is_deleted", "datetime", "close_target"]

df.drop(
    columns=columns_to_drop,
    inplace=True,
)
train_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
validation_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
selected_test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)

In [47]:
train_df.to_csv('/tmp/train_df.csv', index=False, header=False)

train_df_s3_uri = S3Uploader.upload('/tmp/train_df.csv', f's3://{bucket_name}/model_monitor/train_df.csv')

In [32]:
def generate_endpoint_traffic(predictor, data):
    ln = len(data)
    for i in trange(ln):
        row_data = data.iloc[i].values.reshape(1, -1)
        predictions = np.array(
            predictor.predict(row_data), dtype=float
        ).squeeze()

In [33]:
generate_endpoint_traffic(predictor, selected_test_df)

100%|██████████| 100/100 [00:01<00:00, 66.88it/s]


In [34]:
capture_files = utils.get_file_list(bucket_name, data_capture_prefix)

Found 8 files in s3://team1-index-predictor-bucket/data-capture


In [44]:
data_monitor = DefaultModelMonitor(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=10,
    max_runtime_in_seconds=3600,
    sagemaker_session=sm_session,
)

In [49]:
data_baseline_job_name = f"model-monitor-baseline-{strftime('%d-%H-%M-%S', gmtime())}"

data_baseline_job = data_monitor.suggest_baseline(
    baseline_dataset=train_df_s3_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=f"s3://{bucket_name}/model_monitor/baseline_results",
    wait=False,
    logs=False,
    job_name=data_baseline_job_name,
)

INFO:sagemaker:Creating processing-job with name model-monitor-baseline-26-19-08-36


In [51]:
data_baseline_job.wait(logs=False)
data_monitor.describe_latest_baselining_job()

!

{'ProcessingInputs': [{'InputName': 'baseline_dataset_input',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://team1-index-predictor-bucket/model_monitor/train_df.csv/train_df.csv',
    'LocalPath': '/opt/ml/processing/input/baseline_dataset_input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'monitoring_output',
    'S3Output': {'S3Uri': 's3://team1-index-predictor-bucket/model_monitor/baseline_results',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False}]},
 'ProcessingJobName': 'model-monitor-baseline-26-19-08-36',
 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1,
   'InstanceType': 'ml.m5.xlarge',
   'VolumeSizeInGB': 10}},
 'StoppingCondition': {'MaxRuntimeInSeconds': 3600},
 'AppSpecification': {'ImageUri': '048819808253.dkr.ecr.eu-central-1.amazonaws.com/sag

In [62]:
!aws s3 cp s3://{bucket_name}/model_monitor/baseline_results/constraints.json ./tmp/
!aws s3 cp s3://{bucket_name}/model_monitor/baseline_results/statistics.json ./tmp/
!head -15 tmp/constraints.json
!head -15 tmp/statistics.json

download: s3://team1-index-predictor-bucket/model_monitor/baseline_results/constraints.json to tmp/constraints.json
download: s3://team1-index-predictor-bucket/model_monitor/baseline_results/statistics.json to tmp/statistics.json
{
  "version" : 0.0,
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "_c1",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
{
  "version" : 0.0,
  "dataset" : {
    "item_count" : 6908
  },
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Fractional",
    "numerical_statistics" : {
      "common" : {
        "num_present" : 6908,
        "num_missing" : 0
      },
      "mean" : 5362.103575635247,
      "sum" : 3.704141150048828E7,


In [92]:
latest_data_capture_s3_url = utils.get_latest_data_capture_s3_url(bucket_name, data_capture_prefix)

utils.run_model_monitor_job(
    region="eu-central-1",
    instance_type="ml.m5.xlarge",
    role=sagemaker.get_execution_role(),
    data_capture_path=latest_data_capture_s3_url,
    statistics_path=f"s3://{bucket_name}/model_monitor/baseline_results/statistics.json",
    constraints_path=f"s3://{bucket_name}/model_monitor/baseline_results/constraints.json",
    reports_path=f"s3://{bucket_name}/model_monitor/baseline_results/reports/",
    preprocessor_path="./record_preprocessor.py",
    instance_count=1,
    postprocessor_path=None,
    publish_cloudwatch_metrics="Disabled",
    logs=False,
)

Found 11 files in s3://team1-index-predictor-bucket/data-capture
Latest data capture S3 url: s3://team1-index-predictor-bucket/data-capture/index-predictor-endpoint/AllTraffic/2024/06/26/19


INFO:sagemaker:Creating processing-job with name sagemaker-model-monitor-analyzer-2024-06-26-20-55-01-432


...........................................................!

In [94]:
manual_monitoring_job_output_s3_url = utils.get_latest_monitoring_report_s3_url("sagemaker-model-monitor-analyzer")

Latest monitoring report S3 url: s3://team1-index-predictor-bucket/model_monitor/baseline_results/reports//9


In [95]:
!aws s3 ls {manual_monitoring_job_output_s3_url}/

2024-06-26 20:58:55      37229 constraint_violations.json
2024-06-26 20:58:49      24989 constraints.json
2024-06-26 20:58:50     277283 statistics.json


In [103]:
data_mon_schedule_name = "index-predictor-data-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

data_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_mon_schedule_name,
    endpoint_input=predictor.endpoint_name,
    record_preprocessor_script="./record_preprocessor.py",
    output_s3_uri=f"s3://{bucket_name}/model_monitor/baseline_results/reports/",
    statistics=data_monitor.baseline_statistics(),
    constraints=data_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: index-predictor-data-monitor-schedule-2024-06-26-21-24-56


In [108]:
while data_monitor.describe_schedule()["MonitoringScheduleStatus"] != "Scheduled":
    print(f"Waiting until data monitoring schedule status becomes Scheduled")
    time.sleep(3)

data_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:eu-central-1:567821811420:monitoring-schedule/index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
 'MonitoringScheduleName': 'index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2024, 6, 26, 21, 24, 57, 850000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 6, 26, 23, 12, 21, 239000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2024-06-26-21-24-56-894',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'index-predictor-endpoint',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
  'ScheduledTime': datetime.datetime(2024, 6, 26, 23, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 6, 2

In [ ]:
data_monitor.delete_monitoring_schedule(name=data_mon_schedule_name)

In [46]:
%pip install -q jsonlines

import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.model_monitor import (
    DefaultModelMonitor,
    DataCaptureConfig,
    CronExpressionGenerator,
    ModelQualityMonitor,
    EndpointInput,
)
from sagemaker.model_monitor.dataset_format import DatasetFormat
import json
import jsonlines
import utils
from importlib import reload
from tqdm import trange
import numpy as np
import time
import pandas as pd
from time import gmtime, strftime

reload(utils)

Note: you may need to restart the kernel to use updated packages.


<module 'utils' from '/home/sagemaker-user/src/monitoring/utils.py'>

In [4]:
sm_session = sagemaker.Session()

endpoint_name = "index-predictor-endpoint"
feature_group_name = "index-predictor-feature-group-v7"
bucket_name = "team1-index-predictor-bucket"
data_version = "2024-06-26-09-33"

data_capture_prefix = "data-capture"
data_capture_s3_url = f"s3://{bucket_name}/{data_capture_prefix}"

In [5]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer(),
)

### Downloading data from feature store

In [30]:
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sm_session)

query = feature_group.athena_query()

query.run(
    query_string=f"""SELECT * FROM "{query.table_name}" WHERE version = '{data_version}'""",
    output_location=f"s3://{bucket_name}/model_monitor/data/",
)

query.wait()

df = query.as_dataframe()

train_df = df[df["type"] == "train"].copy()
validation_df = df[df["type"] == "validation"].copy()
test_df = df[df["type"] == "test"].copy()

selected_test_df = test_df.copy().sample(n=100, random_state=1)

In [31]:
columns_to_drop = ["type", "version", "write_time", "api_invocation_time", "is_deleted", "datetime", "close_target"]

df.drop(
    columns=columns_to_drop,
    inplace=True,
)
train_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
validation_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)
selected_test_df.drop(
    columns=columns_to_drop,
    inplace=True,
)

In [47]:
train_df.to_csv('/tmp/train_df.csv', index=False, header=False)

train_df_s3_uri = S3Uploader.upload('/tmp/train_df.csv', f's3://{bucket_name}/model_monitor/train_df.csv')

In [32]:
def generate_endpoint_traffic(predictor, data):
    ln = len(data)
    for i in trange(ln):
        row_data = data.iloc[i].values.reshape(1, -1)
        predictions = np.array(
            predictor.predict(row_data), dtype=float
        ).squeeze()

In [33]:
generate_endpoint_traffic(predictor, selected_test_df)

100%|██████████| 100/100 [00:01<00:00, 66.88it/s]


In [34]:
capture_files = utils.get_file_list(bucket_name, data_capture_prefix)

Found 8 files in s3://team1-index-predictor-bucket/data-capture


In [44]:
data_monitor = DefaultModelMonitor(
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=10,
    max_runtime_in_seconds=3600,
    sagemaker_session=sm_session,
)

In [49]:
data_baseline_job_name = f"model-monitor-baseline-{strftime('%d-%H-%M-%S', gmtime())}"

data_baseline_job = data_monitor.suggest_baseline(
    baseline_dataset=train_df_s3_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=f"s3://{bucket_name}/model_monitor/baseline_results",
    wait=False,
    logs=False,
    job_name=data_baseline_job_name,
)

INFO:sagemaker:Creating processing-job with name model-monitor-baseline-26-19-08-36


In [51]:
data_baseline_job.wait(logs=False)
data_monitor.describe_latest_baselining_job()

!

{'ProcessingInputs': [{'InputName': 'baseline_dataset_input',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://team1-index-predictor-bucket/model_monitor/train_df.csv/train_df.csv',
    'LocalPath': '/opt/ml/processing/input/baseline_dataset_input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'monitoring_output',
    'S3Output': {'S3Uri': 's3://team1-index-predictor-bucket/model_monitor/baseline_results',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False}]},
 'ProcessingJobName': 'model-monitor-baseline-26-19-08-36',
 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1,
   'InstanceType': 'ml.m5.xlarge',
   'VolumeSizeInGB': 10}},
 'StoppingCondition': {'MaxRuntimeInSeconds': 3600},
 'AppSpecification': {'ImageUri': '048819808253.dkr.ecr.eu-central-1.amazonaws.com/sag

In [62]:
!aws s3 cp s3://{bucket_name}/model_monitor/baseline_results/constraints.json ./tmp/
!aws s3 cp s3://{bucket_name}/model_monitor/baseline_results/statistics.json ./tmp/
!head -15 tmp/constraints.json
!head -15 tmp/statistics.json

download: s3://team1-index-predictor-bucket/model_monitor/baseline_results/constraints.json to tmp/constraints.json
download: s3://team1-index-predictor-bucket/model_monitor/baseline_results/statistics.json to tmp/statistics.json
{
  "version" : 0.0,
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "_c1",
    "inferred_type" : "Fractional",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
{
  "version" : 0.0,
  "dataset" : {
    "item_count" : 6908
  },
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Fractional",
    "numerical_statistics" : {
      "common" : {
        "num_present" : 6908,
        "num_missing" : 0
      },
      "mean" : 5362.103575635247,
      "sum" : 3.704141150048828E7,


In [92]:
latest_data_capture_s3_url = utils.get_latest_data_capture_s3_url(bucket_name, data_capture_prefix)

utils.run_model_monitor_job(
    region="eu-central-1",
    instance_type="ml.m5.xlarge",
    role=sagemaker.get_execution_role(),
    data_capture_path=latest_data_capture_s3_url,
    statistics_path=f"s3://{bucket_name}/model_monitor/baseline_results/statistics.json",
    constraints_path=f"s3://{bucket_name}/model_monitor/baseline_results/constraints.json",
    reports_path=f"s3://{bucket_name}/model_monitor/baseline_results/reports/",
    preprocessor_path="./record_preprocessor.py",
    instance_count=1,
    postprocessor_path=None,
    publish_cloudwatch_metrics="Disabled",
    logs=False,
)

Found 11 files in s3://team1-index-predictor-bucket/data-capture
Latest data capture S3 url: s3://team1-index-predictor-bucket/data-capture/index-predictor-endpoint/AllTraffic/2024/06/26/19


INFO:sagemaker:Creating processing-job with name sagemaker-model-monitor-analyzer-2024-06-26-20-55-01-432


...........................................................!

In [94]:
manual_monitoring_job_output_s3_url = utils.get_latest_monitoring_report_s3_url("sagemaker-model-monitor-analyzer")

Latest monitoring report S3 url: s3://team1-index-predictor-bucket/model_monitor/baseline_results/reports//9


In [95]:
!aws s3 ls {manual_monitoring_job_output_s3_url}/

2024-06-26 20:58:55      37229 constraint_violations.json
2024-06-26 20:58:49      24989 constraints.json
2024-06-26 20:58:50     277283 statistics.json


In [103]:
data_mon_schedule_name = "index-predictor-data-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

data_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_mon_schedule_name,
    endpoint_input=predictor.endpoint_name,
    record_preprocessor_script="./record_preprocessor.py",
    output_s3_uri=f"s3://{bucket_name}/model_monitor/baseline_results/reports/",
    statistics=data_monitor.baseline_statistics(),
    constraints=data_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: index-predictor-data-monitor-schedule-2024-06-26-21-24-56


In [108]:
while data_monitor.describe_schedule()["MonitoringScheduleStatus"] != "Scheduled":
    print(f"Waiting until data monitoring schedule status becomes Scheduled")
    time.sleep(3)

data_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:eu-central-1:567821811420:monitoring-schedule/index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
 'MonitoringScheduleName': 'index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2024, 6, 26, 21, 24, 57, 850000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 6, 26, 23, 12, 21, 239000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2024-06-26-21-24-56-894',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'index-predictor-endpoint',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'index-predictor-data-monitor-schedule-2024-06-26-21-24-56',
  'ScheduledTime': datetime.datetime(2024, 6, 26, 23, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 6, 2

In [ ]:
data_monitor.delete_monitoring_schedule(name=data_mon_schedule_name)